## Prepare the dataset

In [6]:
import datetime

import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.metrics import mean_squared_error as mse


def download_data():
    uni_ticker = "UNI-USD"
    eth_ticker = "ETH-USD"
    start = datetime.datetime(2019, 1, 1)
    end = datetime.datetime(2024, 4, 1)
    uni = yf.download(uni_ticker, start=start, end=end, interval="1d")
    eth = yf.download(eth_ticker, start=start, end=end, interval="1d")
    uni = uni.reset_index()
    uni.to_csv("uni.csv", index=False)
    eth = eth.reset_index()
    eth.to_csv("eth.csv", index=False)
    return uni, eth


def process_data(uni: pd.DataFrame, eth: pd.DataFrame):
    uni = uni[uni["Open"] < 0.30]
    uni = uni[["Date", "Open"]]
    eth = eth[["Date", "Open"]]

    uni.rename(columns={"Open": "UNI"}, inplace=True)
    eth.rename(columns={"Open": "ETH"}, inplace=True)

    df = pd.merge(uni, eth, on="Date")
    df.dropna(inplace=True)
    df["price"] = df["ETH"] / df["UNI"]
    ret = 100 * (df["price"].pct_change()[1:])
    realized_vol = ret.rolling(5).std()
    realized_vol = pd.DataFrame(realized_vol)
    realized_vol.reset_index(drop=True, inplace=True)
    returns_svm = ret**2  # returns squared
    returns_svm = returns_svm.reset_index()
    X = pd.concat([realized_vol, returns_svm], axis=1, ignore_index=True)
    X = X[4:].copy()
    X = X.reset_index()
    X.drop("index", axis=1, inplace=True)
    X.drop(1, axis=1, inplace=True)
    X.rename(columns={0: "realized_vol", 2: "returns_squared"}, inplace=True)
    X["target"] = X["realized_vol"].shift(-5)
    X.dropna(inplace=True)
    Y = X["target"]
    X.drop("target", axis=1, inplace=True)
    n = 252
    X_train = X.iloc[:-n]
    X_test = X.iloc[-n:]
    Y_train = Y.iloc[:-n]
    Y_test = Y.iloc[-n:]
    return X_train.to_numpy(), X_test.to_numpy(), Y_train.to_numpy(), Y_test.to_numpy()



In [7]:
uni, eth = download_data()
X_train, X_test, Y_train, Y_test = process_data(uni, eth)
print("x-train", X_train.shape)
print("y-train", Y_train.shape)
print("x-test", X_test.shape)
print("y-test", Y_test.shape)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


x-train (1359, 2)
y-train (1359,)
x-test (252, 2)
y-test (252,)


/var/folders/ng/0bmxnvj96057v3jysxjk_9xw0000gn/T/ipykernel_60339/653528502.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth.rename(columns={"Open": "ETH"}, inplace=True)


## Train and transpile the model

In [3]:
import xgboost as xgb
n_estimators = 5  # Increase the number of trees
max_depth = 10  # Increase the maximum depth of each tree

xgb_reg = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth)
xgb_reg.fit(X_train, Y_train)

# save model into json 
from giza.zkcook import serialize_model
serialize_model(xgb_reg, "uniswap_lp_xgboost.json")

In [4]:
# eval
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    # eval metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"R-squared (R²): {r2:.4f}")

    return mae, mse, rmse, r2

mae, mse, rmse, r2 = evaluate_model(xgb_reg, X_test, Y_test)

Mean Absolute Error (MAE): 21.8232
Mean Squared Error (MSE): 3058.2166
Root Mean Squared Error (RMSE): 55.3011
R-squared (R²): 0.0349


In [5]:
# transpile the model into Orion Cairo
! giza transpile uniswap_lp_xgboost.json --output-path uniswap_lp_xgboost 

[giza][2024-06-27 16:37:03.932] No model id provided, checking if model exists ✅
[giza][2024-06-27 16:37:03.934] Model name is: uniswap_lp_xgboost
[giza][2024-06-27 16:37:04.184] Model already exists, using existing model ✅ 
[giza][2024-06-27 16:37:04.185] Model found with id -> 862! ✅
[giza][2024-06-27 16:37:04.811] Version Created with id -> 2! ✅
[giza][2024-06-27 16:37:04.812] Sending model for transpilation ✅ 
[giza][2024-06-27 16:37:25.658] Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
⠙ Transpiling Model...
[giza][2024-06-27 16:37:26.294] Downloading model ✅
[giza][2024-06-27 16:37:26.301] model saved at: uniswap_lp_xgboost
                  Model                   
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ id  ┃ name               ┃ description ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 862 │ uniswap_lp_xgboost │             │
└─────┴────────────────────┴─────────────┘
                                    Version                         

In [1]:
MODEL_ID = 862
VERSION_ID = 2

In [7]:
# create giza endpoint
! giza endpoints deploy --model-id {MODEL_ID} --version-id {VERSION_ID}

▰▱▱▱▱▱▱ Creating endpoint!t!
[giza][2024-06-27 16:38:03.812] Endpoint is successful ✅
[giza][2024-06-27 16:38:03.817] Endpoint created with id -> 410 ✅
[giza][2024-06-27 16:38:03.818] Endpoint created with endpoint URL: https://endpoint-selimsheker-862-2-f14d7c7f-7i3yxzspbq-ew.a.run.app 🎉


In [2]:
ENDPOINT_ID = 410

# Run verifiable inference using giza sdk

In [9]:
import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

from giza.agents.model import GizaModel

In [10]:
def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={"input": input}, verifiable=True, model_category="XGB"
    )

    return result, proof_id


def execution():
    input = X_test[1, :]

    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(f"Predicted value for input {input.flatten()[0]} is {result}")

    return result, proof_id

In [11]:
result, proof_id = execution()
print(f"Proof ID: {proof_id}")

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Predicted value for input 22.38294835718384 is 16.21992
Proof ID: 786477d48e784bf79c99c07c2c8f0ab2


In [12]:
PROOF_ID = "786477d48e784bf79c99c07c2c8f0ab2"

# Verify the proof

In [28]:
# get proof
! giza endpoints get-proof --endpoint-id {ENDPOINT_ID} --proof-id "{PROOF_ID}"

[giza][2024-06-27 15:52:45.667] Getting proof from endpoint 407 ✅ 
┏━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ id   ┃ job_id ┃ proving_t… ┃ cairo_exe… ┃ metrics   ┃ created_d… ┃ request_… ┃
┡━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 1271 │ 1463   │            │            │ {'provin… │ 2024-06-27 │           │
│      │        │            │            │ 16.2721}  │ 12:50:47.… │           │
└──────┴────────┴────────────┴────────────┴───────────┴────────────┴───────────┘


In [29]:
# download proof
! giza endpoints download-proof --endpoint-id {ENDPOINT_ID} --proof-id "{PROOF_ID}" --output-path uniswap_lp_xgboost_proof.proof

[giza][2024-06-27 15:53:26.844] Getting proof from endpoint 407 ✅ 
[giza][2024-06-27 15:53:28.049] Proof downloaded to uniswap_lp_xgboost_proof.proof ✅


In [30]:
# verify
!giza verify --proof-id 1271

[giza][2024-06-27 15:53:39.007] Verifying proof...
[giza][2024-06-27 15:53:40.425] Verification result: True
[giza][2024-06-27 15:53:40.426] Verification time: 0.458288615


## Create giza agent

In [ ]:
# ! giza agents create --endpoint-id 401 --name uniswap_lp_agent --description uniswap_lp_agent

In [34]:
import argparse
import logging
import os
import pprint
from logging import getLogger

import numpy as np
from dotenv import find_dotenv, load_dotenv
from giza.agents import AgentResult, GizaAgent

from giza_helpers.addresses import ADDRESSES
from giza_helpers.lp_tools import get_tick_range
from giza_helpers.uni_helpers import (approve_token, check_allowance, close_position,
                         get_all_user_positions, get_mint_params)


load_dotenv(find_dotenv())

os.environ["GIZA-AGENT-TEST-1_PASSPHRASE"] = os.environ.get("GIZA-AGENT-TEST-1_PASSPHRASE")
sepolia_rpc_url = os.environ.get("SEPOLIA_RPC_URL")

logging.basicConfig(level=logging.INFO)


In [45]:
def process_data(realized_vol: float, dec_price_change: float):
    pct_change_sq = (100 * dec_price_change) ** 2
    print(realized_vol, pct_change_sq)
    X = np.array([realized_vol, pct_change_sq])
    return X


def get_data():
    # TODO: implement fetching onchain or from some other source
    # hardcoding the values for now
    realized_vol = 4.20
    dec_price_change = 0.1
    return realized_vol, dec_price_change


def create_agent(
    model_id: int, version_id: int, chain: str, contracts: dict, account: str
):
    """
    Create a Giza agent for the volatility prediction model
    """
    agent = GizaAgent(
        contracts=contracts,
        id=model_id,
        version_id=version_id,
        chain=chain,
        account=account,
    )
    return agent


def predict(agent: GizaAgent, X: np.ndarray):
    """
    Predict the next day volatility.

    Args:
        X (np.ndarray): Input to the model.

    Returns:
        int: Predicted value.
    """
    print(X)
    prediction = agent.predict(input_feed={"val": X}, verifiable=True, job_size="XL")
    return prediction


def get_pred_val(prediction: AgentResult):
    """
    Get the value from the prediction.

    Args:
        prediction (dict): Prediction from the model.

    Returns:
        int: Predicted value.
    """
    # This will block the executon until the prediction has generated the proof
    # and the proof has been verified
    return prediction.value[0][0]


def rebalance_lp(
    tokenA_amount: int,
    tokenB_amount: int,
    pred_model_id: int,
    pred_version_id: int,
    account="dev",
    chain=f"ethereum:sepolia:{sepolia_rpc_url}",
    nft_id=None,
):
    logger = getLogger("agent_logger")
    nft_manager_address = ADDRESSES["NonfungiblePositionManager"][11155111]
    tokenA_address = ADDRESSES["UNI"][11155111]
    tokenB_address = ADDRESSES["WETH"][11155111]
    pool_address = "0x287B0e934ed0439E2a7b1d5F0FC25eA2c24b64f7"
    user_address = "0xCBB090699E0664f0F6A4EFbC616f402233718152"
    pool_fee = 3000
    logger.info("Fetching input data")
    realized_vol, dec_price_change = get_data()
    logger.info(f"Input data: {realized_vol}, {dec_price_change}")
    X = process_data(realized_vol, dec_price_change)
    contracts = {
        "nft_manager": nft_manager_address,
        "tokenA": tokenA_address,
        "tokenB": tokenB_address,
        "pool": pool_address,
    }
    agent = create_agent(
        model_id=pred_model_id,
        version_id=pred_version_id,
        chain=chain,
        contracts=contracts,
        account=account,
    )
    result = predict(agent, X)
    predicted_value = get_pred_val(result)
    logger.info(f"Result: {result}")
    with agent.execute() as contracts:
        logger.info("Executing contract")
        if nft_id is None:
            positions = [
                max(get_all_user_positions(contracts.nft_manager, user_address))
            ]
        else:
            positions = [nft_id]
        logger.info(f"Closing the following positions {positions}")
        for nft_id in positions:
            close_position(user_address, contracts.nft_manager, nft_id)
        logger.info("Calculating mint params...")
        _, curr_tick, _, _, _, _, _ = contracts.pool.slot0()
        if not check_allowance(
            contracts.tokenA, nft_manager_address, account, tokenA_amount
        ):
            approve_token(contracts.tokenA, nft_manager_address, tokenA_amount)
        if not check_allowance(
            contracts.tokenB, nft_manager_address, account, tokenB_amount
        ):
            approve_token(contracts.tokenB, nft_manager_address, tokenB_amount)
        tokenA_decimals = contracts.tokenA.decimals()
        tokenB_decimals = contracts.tokenB.decimals()
        predicted_value = predicted_value / 100 * 1.96  # convert to decimal %
        lower_tick, upper_tick = get_tick_range(
            curr_tick, predicted_value, tokenA_decimals, tokenB_decimals, pool_fee
        )
        mint_params = get_mint_params(
            user_address,
            contracts.tokenA.address,
            contracts.tokenB.address,
            tokenA_amount,
            tokenB_amount,
            pool_fee,
            lower_tick,
            upper_tick,
        )
        # step 5: mint new position
        logger.info("Minting new position...")
        contract_result = contracts.nft_manager.mint(mint_params)
        logger.info("SUCCESSFULLY MINTED A POSITION")
        logger.info("Contract executed")

    logger.info(f"Contract result: {contract_result}")
    pprint.pprint(contract_result.__dict__)
    logger.info("Finished")


In [ ]:

tokenA_amount = 500
tokenB_amount = 500

rebalance_lp(tokenA_amount, tokenB_amount, MODEL_ID, VERSION_ID, account="giza-agent-test-1")

## Benchmarks

In [1]:
## xgboost transpile and prediction stats

# - n_estimators = 5, max_depth = 10
#     - transpile from json: ~1sec
#     - transpile from onnx: ~1min
#     - generate proof: ~5sec
#     - verify proof: ~1sec

# - n_estimators = 50, max_depth = 50
#     - transpile: out of memory
